In [ ]:
%pip install yfinance

In [ ]:
import yfinance as yf
from datetime import datetime, timedelta

# Define the ticker symbol for Reliance Industries (RIL) on NSE
ticker_symbol = "RELIANCE.NS"

# Calculate the start date (5 years ago from today)
end_date = datetime.today()
start_date = end_date - timedelta(days=10*365) # Approximating 10 years

# Fetch the data
reliance_data = yf.download(ticker_symbol, start=start_date, end=end_date)

# Display the first few rows of the data
display(reliance_data.head())

# Save the data to a CSV file
reliance_data.to_csv('reliance_stock_data.csv')
print("\nReliance stock data saved to 'reliance_stock_data.csv'")

/tmp/ipython-input-937205592.py:12: FutureWarning:

YF.download() has changed argument auto_adjust default to True

[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume
Ticker,RELIANCE.NS,RELIANCE.NS,RELIANCE.NS,RELIANCE.NS,RELIANCE.NS
Date,,,,,
2015-09-14,187.182373,187.583241,184.593002,186.727335,7964072
2015-09-15,187.864975,188.515037,185.394776,186.781557,11509788
2015-09-16,189.067551,190.107633,187.474926,188.970046,13424811
2015-09-18,194.018784,194.907177,190.064299,190.345980,34261453
2015-09-21,190.270142,192.458645,189.825938,190.681844,12732187



Reliance stock data saved to 'reliance_stock_data.csv'


In [ ]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

# Use the 'cleaned_reliance_data' DataFrame which is already available
# Select the 'Close' price column and convert it to a numpy array
close_prices = cleaned_reliance_data['Close'].values.reshape(-1, 1)

# Scale the data using MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(close_prices)

# Create the training data set
X = []
Y = []
time_step = 60

for i in range(time_step, len(scaled_data)):
    X.append(scaled_data[i-time_step:i, 0])
    Y.append(scaled_data[i, 0])

# Convert X and Y to numpy arrays
X, Y = np.array(X), np.array(Y)

# Reshape X for the LSTM model (samples, time_steps, features)
X = np.reshape(X, (X.shape[0], X.shape[1], 1))

print("Shape of X:", X.shape)
print("Shape of Y:", Y.shape)

Shape of X: (2405, 60, 1)
Shape of Y: (2405,)


In [ ]:
# Split the data into training, validation, and testing sets
train_size = int(len(X) * 0.8)  # 80% for training
val_size = int(len(X) * 0.1)   # 10% for validation
test_size = len(X) - train_size - val_size # Remaining 10% for testing

X_train, X_val, X_test = X[:train_size], X[train_size:train_size+val_size], X[train_size+val_size:]
Y_train, Y_val, Y_test = Y[:train_size], Y[train_size:train_size+val_size], Y[train_size+val_size:]

print("Shape of X_train:", X_train.shape)
print("Shape of Y_train:", Y_train.shape)
print("Shape of X_val:", X_val.shape)
print("Shape of Y_val:", Y_val.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of Y_test:", Y_test.shape)

Shape of X_train: (1924, 60, 1)
Shape of Y_train: (1924,)
Shape of X_val: (240, 60, 1)
Shape of Y_val: (240,)
Shape of X_test: (241, 60, 1)
Shape of Y_test: (241,)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

# Build the LSTM model
model = Sequential()

# Add LSTM layers with Dropout
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(Dropout(0.2))

model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(units=50))
model.add(Dropout(0.2))

# Add the output layer
model.add(Dense(units=1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Print the model summary
model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_3 (LSTM)                   │ (None, 60, 50)         │        10,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 60, 50)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_4 (LSTM)                   │ (None, 60, 50)         │        20,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 60, 50)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_5 (LSTM)                   │ (None, 60, 50)         │        20,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 60, 50)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_6 (LSTM)                   │ (None, 50)             │        20,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 50)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 71,051 (277.54 KB)

 Trainable params: 71,051 (277.54 KB)

 Non-trainable params: 0 (0.00 B)

In [31]:
# Train the model
history = model.fit(X_train, Y_train, epochs=100, batch_size=32, validation_data=(X_val, Y_val))

Epoch 1/100
61/61 ━━━━━━━━━━━━━━━━━━━━ 8s 135ms/step - loss: 6.3676e-04 - val_loss: 0.0016
Epoch 2/100
61/61 ━━━━━━━━━━━━━━━━━━━━ 9s 117ms/step - loss: 5.4810e-04 - val_loss: 0.0040
Epoch 3/100
61/61 ━━━━━━━━━━━━━━━━━━━━ 10s 113ms/step - loss: 6.3869e-04 - val_loss: 0.0030
Epoch 4/100
61/61 ━━━━━━━━━━━━━━━━━━━━ 11s 119ms/step - loss: 5.7781e-04 - val_loss: 0.0044
Epoch 5/100
61/61 ━━━━━━━━━━━━━━━━━━━━ 11s 125ms/step - loss: 5.7210e-04 - val_loss: 0.0017
Epoch 6/100
61/61 ━━━━━━━━━━━━━━━━━━━━ 11s 133ms/step - loss: 5.8209e-04 - val_loss: 0.0022
Epoch 7/100
61/61 ━━━━━━━━━━━━━━━━━━━━ 7s 121ms/step - loss: 5.6514e-04 - val_loss: 0.0020
Epoch 8/100
61/61 ━━━━━━━━━━━━━━━━━━━━ 10s 114ms/step - loss: 8.1248e-04 - val_loss: 0.0028
Epoch 9/100
61/61 ━━━━━━━━━━━━━━━━━━━━ 10s 113ms/step - loss: 6.1548e-04 - val_loss: 0.0050
Epoch 10/100
61/61 ━━━━━━━━━━━━━━━━━━━━ 11s 132ms/step - loss: 5.5663e-04 - val_loss: 0.0027
Epoch 11/100
61/61 ━━━━━━━━━━━━━━━━━━━━ 8s 135ms/step - loss: 6.6055e-04 - val_los

In [34]:
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from math import sqrt
import plotly.graph_objects as go

# Make predictions on the test data
predictions = model.predict(X_test)

# Inverse transform the predictions and the actual test data
predictions = scaler.inverse_transform(predictions)
Y_test_inverse = scaler.inverse_transform(Y_test.reshape(-1, 1))

# Calculate RMSE
rmse = sqrt(mean_squared_error(Y_test_inverse, predictions))
print(f"Root Mean Squared Error (RMSE): {rmse}")

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
Root Mean Squared Error (RMSE): 65.10898200008818


In [36]:
# Create an interactive plot using Plotly
fig = go.Figure()

fig.add_trace(go.Scatter(x=list(range(len(Y_test_inverse))), y=Y_test_inverse.flatten(), mode='lines', name='Actual Price'))
fig.add_trace(go.Scatter(x=list(range(len(predictions))), y=predictions.flatten(), mode='lines', name='Predicted Price'))

fig.update_layout(
    title='Reliance Stock Price Prediction (Test Set)',
    xaxis_title='Time Step',
    yaxis_title='Close Price INR (₹)',
    legend_title='Price Type'
)

fig.show()

In [57]:
!git config --global user.name "BhavikBuchke"
!git config --global user.email "bbuchke@gmail.com"


In [58]:
%cd /content/drive/MyDrive/LSTM.ipynb


[Errno 2] No such file or directory: '/content/drive/MyDrive/LSTM.ipynb'
/content/Stock-price-Prediction
